In [ ]:
from dash import Dash
import dash_leaflet as dl

app = Dash()
app.layout = dl.Map(dl.TileLayer(), center=[56,10], zoom=6, style={'height': '50vh'})

if __name__ == '__main__':
    app.run_server()

In [ ]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import dash_leaflet as dl
import pandas as pd
import geopandas as gpd
from shapely import wkt
from pyproj import Transformer
from shapely.geometry import LineString, MultiLineString

# Data imports
vz_moped = pd.read_csv('../Output/annualized_statistics.csv')

# Data manipulation

# Changing null values to "N/As"
vz_moped['component_subtype'] = vz_moped['component_subtype'].fillna("N/A")
vz_moped['component_work_types'] = vz_moped['component_work_types'].fillna("N/A")

# Converting to datetime
vz_moped['substantial_completion_date'] = pd.to_datetime(vz_moped['substantial_completion_date'])

# Creating completion year variable
vz_moped['completion_year'] = vz_moped['substantial_completion_date'].dt.year

# Flag for involving fatality
# 1 if pre/post fatal crash rate is not null
vz_moped['component_had_fatal_crash'] = (vz_moped['pre_annualized_fatal_crash_rate'] > 0) | (vz_moped['post_annualized_fatal_crash_rate'] > 0)
vz_moped['component_had_fatal_crash'] = vz_moped['component_had_fatal_crash'].apply(lambda x: "Yes" if x else "No")

# Assuming vz_moped is your pandas DataFrame and has been defined earlier
vz_moped['line_geometry'] = vz_moped['line_geometry'].apply(wkt.loads)

# Simplify the geometries for performance
SIMPLIFY_TOLERANCE = 0.0001  # Set a proper tolerance value for simplification
vz_moped['line_geometry'] = vz_moped['line_geometry'].apply(lambda geom: geom.simplify(SIMPLIFY_TOLERANCE))

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

transformer = Transformer.from_crs("EPSG:32614", "EPSG:4326", always_xy=True)

# def transform_coordinates(geom):

#     if geom.geom_type == 'LineString':
#         transformed = LineString([transformer.transform(x, y)[::-1] for x, y in geom.coords])
#     elif geom.geom_type == 'MultiLineString':
#         transformed = MultiLineString([
#             LineString([transformer.transform(x, y)[::-1] for x, y in line.coords])
#             for line in geom.geoms
#         ])
#     else:
#         raise ValueError(f"Unsupported geometry type: {geom.geom_type}")
    
#     return transformed

# def swap_coordinates(coords):
#     return [[y, x] for x, y in coords]

# def create_geojson(filtered_df):
#     if filtered_df.empty:
#         return {}

#     gdf = gpd.GeoDataFrame(filtered_df, geometry='line_geometry')
    
#     for column in gdf.select_dtypes(include=['datetimetz', 'datetime64']).columns:
#         gdf[column] = gdf[column].astype(str)
    
#     # Transform the coordinates
#     gdf['line_geometry'] = gdf['line_geometry'].apply(transform_coordinates)

#     gdf['line_geometry'] = gdf['line_geometry'].apply(lambda geom: swap_coordinates(geom))
        
#     geojson_data = gdf.__geo_interface__

#     return geojson_data

# Define a function to transform coordinates using a transformer
def transform_coordinates(geom, transformer):
    if geom.geom_type == 'Point':
        x, y = geom.x, geom.y
        return Point(*transformer.transform(x, y)[::-1])
    elif geom.geom_type == 'LineString':
        return LineString([transformer.transform(x, y)[::-1] for x, y in geom.coords])
    elif geom.geom_type == 'MultiLineString':
        return MultiLineString([
            LineString([transformer.transform(x, y)[::-1] for x, y in line.coords])
            for line in geom.geoms
        ])
    else:
        raise ValueError(f"Unsupported geometry type: {geom.geom_type}")

# Define a function to create GeoJSON data
def create_geojson(filtered_df):
    if filtered_df.empty:
        return {}

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(filtered_df, geometry='line_geometry')
    
    # Convert datetime columns to string
    for column in gdf.select_dtypes(include=['datetimetz', 'datetime64']).columns:
        gdf[column] = gdf[column].astype(str)

    # Set up the transformer
    transformer = Transformer.from_crs("epsg:32614", "epsg:4326", always_xy=True)

    # Transform the coordinates
    gdf['line_geometry'] = gdf['line_geometry'].apply(lambda geom: transform_coordinates(geom, transformer))
    
    # Get GeoJSON data
    geojson_data = gdf.__geo_interface__

    return geojson_data


def create_card(title, dropdown_id, options):
    return dbc.Card(
        [
            dbc.CardHeader(title),
            dbc.CardBody(
                dcc.Dropdown(id=dropdown_id, options=[{'label': i, 'value': i} for i in options], multi=True)
            )
        ]
    )

app.layout = dbc.Container([
    dbc.Row(
        dbc.Col(html.H1("Map Visualization", className="app_title"),
                width=12),
        className="justify-content-center"
    ),
    dbc.Row(
        [
            dbc.Col(
                create_card("Completion year", "dropdown_year", sorted(vz_moped['completion_year'].unique())),
                className="navigation_row_col_style"
            ),
            dbc.Col(
                create_card("Component name", "dropdown_component_name", vz_moped['component_name'].unique()),
                className="navigation_row_col_style"
            ),
            dbc.Col(
                create_card("Component sub-type", "dropdown_component_subtype", vz_moped['component_subtype'].unique()),
                className="navigation_row_col_style"
            ),
            dbc.Col(
                create_card("Component work-type", "dropdown_work_type", vz_moped['component_work_types'].unique()),
                className="navigation_row_col_style"
            ),
            dbc.Col(
                create_card("Fatal crash", "dropdown_fatal_crash", vz_moped['component_had_fatal_crash'].unique()),
                className="navigation_row_col_style"
            )
        ],
        className="navigation_row"
    ),
    dbc.Row(
        dbc.Col(
            dl.Map(center=[30.2672, -97.7431], zoom=12, children=[
                # dl.TileLayer(),  # Comment out the background tile layer
                dl.GeoJSON(id='geojson', options=dict(style={'color': 'red', 'weight': 2})),
                dl.Marker(position=[30.2672, -97.7431], children=[
                    dl.Tooltip("Center of Austin")
                ])
            ], style={'width': '100%', 'height': '500px'}),
            width=12
        )
    )
], fluid=True, className="custom_container")

@app.callback(
    Output('geojson', 'data'),
    [
        Input('dropdown_year', 'value'),
        Input('dropdown_component_name', 'value'),
        Input('dropdown_component_subtype', 'value'),
        Input('dropdown_work_type', 'value'),
        Input('dropdown_fatal_crash', 'value')
    ]
)
def update_map(dropdown_year, dropdown_component_name, dropdown_component_subtype,
               dropdown_work_type, dropdown_fatal_crash):
    # Base DataFrame
    filtered_df = vz_moped

    # Convert single values to list to avoid TypeError
    dropdown_year = [dropdown_year] if dropdown_year and not isinstance(dropdown_year, list) else dropdown_year
    dropdown_component_name = [dropdown_component_name] if dropdown_component_name and not isinstance(dropdown_component_name, list) else dropdown_component_name
    dropdown_component_subtype = [dropdown_component_subtype] if dropdown_component_subtype and not isinstance(dropdown_component_subtype, list) else dropdown_component_subtype
    dropdown_work_type = [dropdown_work_type] if dropdown_work_type and not isinstance(dropdown_work_type, list) else dropdown_work_type
    dropdown_fatal_crash = [dropdown_fatal_crash] if dropdown_fatal_crash and not isinstance(dropdown_fatal_crash, list) else dropdown_fatal_crash

    # Apply filters
    if dropdown_year:
        filtered_df = filtered_df[filtered_df['completion_year'].isin(dropdown_year)]
    if dropdown_component_name:
        filtered_df = filtered_df[filtered_df['component_name'].isin(dropdown_component_name)]
    if dropdown_component_subtype:
        filtered_df = filtered_df[filtered_df['component_subtype'].isin(dropdown_component_subtype)]
    if dropdown_work_type:
        filtered_df = filtered_df[filtered_df['component_work_types'].isin(dropdown_work_type)]
    if dropdown_fatal_crash:
        filtered_df = filtered_df[filtered_df['component_had_fatal_crash'].isin(dropdown_fatal_crash)]

    # Create GeoJSON data
    geojson_data = create_geojson(filtered_df)
    
    # print(geojson_data)  # Debug: Ensure GeoJSON data is as expected
    
    return geojson_data

if __name__ == '__main__':
    app.run_server(debug=True, port=8200)

In [12]:
import dash
from dash import html
import dash_leaflet as dl
import random

# Generate random LineString data for Austin
def generate_random_linestring():
    return {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [random.uniform(-97.8, -97.6), random.uniform(30.2, 30.4)]
                for _ in range(3)
            ]
        }
    }

# Generate random MultiLineString data for Austin
def generate_random_multilinestring():
    return {
        "type": "Feature",
        "geometry": {
            "type": "MultiLineString",
            "coordinates": [
                [
                    [random.uniform(-97.8, -97.6), random.uniform(30.2, 30.4)]
                    for _ in range(2)
                ]
                for _ in range(2)
            ]
        }
    }

# Create GeoJSON data
geojson_data = {
    "type": "FeatureCollection",
    "features": [generate_random_linestring() for _ in range(5)] + 
                [generate_random_multilinestring() for _ in range(3)]
}

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    dl.Map([
        dl.TileLayer(),
        dl.GeoJSON(data=geojson_data, id="geojson"),
    ], center=[30.2672, -97.7431], zoom=11, style={'height': '100vh'})
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8888)

In [3]:
geojson_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.71770054103663, 30.27072907723563],
     [-97.74690621476353, 30.327153105648588],
     [-97.65448122689517, 30.264161552740713]]}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.7607663356564, 30.219291808377356],
     [-97.78372264065626, 30.278685135086427],
     [-97.7732619079807, 30.200643544763537]]}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.71663909535764, 30.361975473568542],
     [-97.60339244057737, 30.214671512802923],
     [-97.77768130366292, 30.30106445712947]]}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.66191278756088, 30.26942892015071],
     [-97.79672394586069, 30.372557586763357],
     [-97.72588634254944, 30.299279702162508]]}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.74

In [14]:
import dash
from dash import html
import dash_leaflet as dl

# Function to flip coordinates
def flip_coordinates(coord_list):
    return [tuple(reversed(coord)) for coord in coord_list]

# Your GeoJSON data with flipped coordinates
geojson_data2 = {
    'type': 'FeatureCollection',
    'features': [
        {
            'id': '0',
            'type': 'Feature',
            'properties': {'name': 'Line 1'},
            'geometry': {
                'type': 'LineString',
                'coordinates': flip_coordinates(((30.267724296999987, -97.734226243), (30.267711988999988, -97.73422780000001), (30.267700182, -97.734232092), (30.267689328999996, -97.734238955), (30.267679845999997, -97.734248126), (30.26767209800001, -97.734259252), (30.267666384, -97.734271906), (30.267662921999996, -97.73428560100001), (30.26766184600001, -97.73429981), (30.267663195999987, -97.734313989), (30.267666922, -97.734327591), (30.267672879999985, -97.73434009500001), (30.267680840999997, -97.73435102), (30.267690497999993, -97.734359945), (30.267701482000003, -97.734366528), (30.267713368999992, -97.734370517), (30.267725702999996, -97.734371757), (30.26773801099999, -97.7343702), (30.267749818, -97.734365908), (30.267760670999998, -97.734359045), (30.267770154, -97.734349874), (30.267777902, -97.734338748), (30.267783616000003, -97.73432609400001), (30.26778707799999, -97.734312399), (30.267788154, -97.73429819), (30.267786803999986, -97.73428401100001), (30.267783078000004, -97.734270409), (30.26777712, -97.734257905), (30.267769158999993, -97.73424698), (30.267759501999993, -97.73423805500002), (30.26774851799999, -97.734231472), (30.267736631, -97.734227483), (30.267724296999987, -97.734226243)))
            },
        },
        {
            'id': '1',
            'type': 'Feature',
            'properties': {'name': 'Line 2'},
            'geometry': {
                'type': 'LineString',
                'coordinates': flip_coordinates(((30.266570295999994, -97.733783244), (30.266557989000002, -97.7337848), (30.266546181999995, -97.733789093), (30.266535327999996, -97.73379595600001), (30.266525846, -97.733805127), (30.26651809799999, -97.733816253), (30.266512384000002, -97.73382890600001), (30.26650892199999, -97.733842601), (30.266507845999993, -97.73385681), (30.26650919700001, -97.73387098900001), (30.266512921999997, -97.73388459100002), (30.266518880000003, -97.733897095), (30.266526841000008, -97.733908019), (30.26653649800001, -97.733916944), (30.266547482, -97.733923528), (30.266559368999992, -97.733927516), (30.266571703999993, -97.733928756), (30.266584010999996, -97.73392720000001), (30.266595818, -97.733922907), (30.266606671999998, -97.733916044), (30.266616154, -97.733906873), (30.266623902000003, -97.733895747), (30.266629616, -97.733883094), (30.266633078, -97.733869399), (30.266634154000002, -97.73385519), (30.266632802999993, -97.733841011), (30.266629077999998, -97.733827409), (30.266623120000013, -97.73381490500002), (30.266615159, -97.733803981), (30.266605502000008, -97.733795056), (30.266594517999987, -97.733788472), (30.266582631000002, -97.733784484), (30.266570295999994, -97.733783244)))
            },
        }
    ]
}

# Define the style function
style = dict(weight=4, opacity=0.8, color='blue')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    dl.Map([
        dl.TileLayer(),
        dl.GeoJSON(data=geojson_data2, 
                   id="geojson",
                   style=style,
                   hoverStyle=dict(weight=5, color='red', dashArray=''),
                   zoomToBounds=True,
                   zoomToBoundsOnClick=True,
                   options=dict(style=style)),
        dl.Tooltip(id="tooltip"),
    ], style={'height': '100vh'})
])

@app.callback(
    dash.Output("tooltip", "children"),
    [dash.Input("geojson", "hover_feature")]
)
def update_tooltip(feature):
    if feature is not None:
        return feature["properties"]["name"]
    return ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8111)

In [15]:
geojson_data2

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'name': 'Line 1'},
   'geometry': {'type': 'LineString',
    'coordinates': [(-97.734226243, 30.267724296999987),
     (-97.73422780000001, 30.267711988999988),
     (-97.734232092, 30.267700182),
     (-97.734238955, 30.267689328999996),
     (-97.734248126, 30.267679845999997),
     (-97.734259252, 30.26767209800001),
     (-97.734271906, 30.267666384),
     (-97.73428560100001, 30.267662921999996),
     (-97.73429981, 30.26766184600001),
     (-97.734313989, 30.267663195999987),
     (-97.734327591, 30.267666922),
     (-97.73434009500001, 30.267672879999985),
     (-97.73435102, 30.267680840999997),
     (-97.734359945, 30.267690497999993),
     (-97.734366528, 30.267701482000003),
     (-97.734370517, 30.267713368999992),
     (-97.734371757, 30.267725702999996),
     (-97.7343702, 30.26773801099999),
     (-97.734365908, 30.267749818),
     (-97.734359045, 30.267760670999998),
     (-9

In [13]:
geojson_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.67197008367589, 30.31313647807175],
     [-97.6468067060936, 30.3881805805385],
     [-97.63487557391628, 30.397073314012133]]}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.60078952304369, 30.390446321318947],
     [-97.78660977350906, 30.357661170552728],
     [-97.64517563622381, 30.242316143203613]]}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.61906468996452, 30.295855558424055],
     [-97.7555836221705, 30.21321739952658],
     [-97.66071581936151, 30.212614834627324]]}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.6434718322127, 30.20118635171394],
     [-97.66992835929062, 30.236130911370825],
     [-97.79031239674826, 30.234233794422973]]}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-97.62578

chatgpt

In [16]:
import dash
from dash import html
import dash_leaflet as dl

# Your GeoJSON data with coordinates in [longitude, latitude] format
geojson_data2 = {
    'type': 'FeatureCollection',
    'features': [
        {
            'id': '0',
            'type': 'Feature',
            'properties': {'name': 'Line 1'},
            'geometry': {
                'type': 'LineString',
                'coordinates': [
                    (-97.734226243, 30.267724296999987),
                    (-97.73422780000001, 30.267711988999988),
                    (-97.734232092, 30.267700182),
                    (-97.734238955, 30.267689328999996),
                    (-97.734248126, 30.267679845999997),
                    (-97.734259252, 30.26767209800001),
                    (-97.734271906, 30.267666384),
                    (-97.73428560100001, 30.267662921999996),
                    (-97.73429981, 30.26766184600001),
                    (-97.734313989, 30.267663195999987),
                    (-97.734327591, 30.267666922),
                    (-97.73434009500001, 30.267672879999985),
                    (-97.73435102, 30.267680840999997),
                    (-97.734359945, 30.267690497999993),
                    (-97.734366528, 30.267701482000003),
                    (-97.734370517, 30.267713368999992),
                    (-97.734371757, 30.267725702999996),
                    (-97.7343702, 30.26773801099999),
                    (-97.734365908, 30.267749818),
                    (-97.734359045, 30.267760670999998),
                    (-97.734349874, 30.267770154),
                    (-97.734338748, 30.267777902),
                    (-97.73432609400001, 30.267783616000003),
                    (-97.734312399, 30.26778707799999),
                    (-97.73429819, 30.267788154),
                    (-97.73428401100001, 30.267786803999986),
                    (-97.734270409, 30.267783078000004),
                    (-97.734257905, 30.26777712),
                    (-97.73424698, 30.267769158999993),
                    (-97.73423805500002, 30.267759501999993),
                    (-97.734231472, 30.26774851799999),
                    (-97.734227483, 30.267736631),
                    (-97.734226243, 30.267724296999987)
                ]
            },
        },
        {
            'id': '1',
            'type': 'Feature',
            'properties': {'name': 'Line 2'},
            'geometry': {
                'type': 'LineString',
                'coordinates': [
                    (-97.733783244, 30.266570295999994),
                    (-97.7337848, 30.266557989000002),
                    (-97.733789093, 30.266546181999995),
                    (-97.73379595600001, 30.266535327999996),
                    (-97.733805127, 30.266525846),
                    (-97.733816253, 30.26651809799999),
                    (-97.73382890600001, 30.266512384000002),
                    (-97.733842601, 30.26650892199999),
                    (-97.73385681, 30.266507845999993),
                    (-97.73387098900001, 30.26650919700001),
                    (-97.73388459100002, 30.266512921999997),
                    (-97.733897095, 30.266518880000003),
                    (-97.733908019, 30.266526841000008),
                    (-97.733916944, 30.26653649800001),
                    (-97.733923528, 30.266547482),
                    (-97.733927516, 30.266559368999992),
                    (-97.733928756, 30.266571703999993),
                    (-97.73392720000001, 30.266584010999996),
                    (-97.733922907, 30.266595818),
                    (-97.733916044, 30.266606671999998),
                    (-97.733906873, 30.266616154),
                    (-97.733895747, 30.266623902000003),
                    (-97.733883094, 30.266629616),
                    (-97.733869399, 30.266633078),
                    (-97.73385519, 30.266634154000002),
                    (-97.733841011, 30.266632802999993),
                    (-97.733827409, 30.266629077999998),
                    (-97.73381490500002, 30.266623120000013),
                    (-97.733803981, 30.266615159),
                    (-97.733795056, 30.266605502000008),
                    (-97.733788472, 30.266594517999987),
                    (-97.733784484, 30.266582631000002),
                    (-97.733783244, 30.266570295999994)
                ]
            },
        }
    ]
}

# Define the style function
style = dict(weight=4, opacity=0.8, color='blue')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    dl.Map([
        dl.TileLayer(),
        dl.GeoJSON(data=geojson_data2,  # Use the correct GeoJSON variable
                   id="geojson",
                   style=style,
                   hoverStyle=dict(weight=5, color='red', dashArray=''),
                   zoomToBounds=True,
                   zoomToBoundsOnClick=True,
                   options=dict(style=style)),
        dl.Tooltip(id="tooltip"),
    ], style={'height': '100vh'})
])

@app.callback(
    dash.Output("tooltip", "children"),
    [dash.Input("geojson", "hover_feature")]
)
def update_tooltip(feature):
    if feature is not None:
        return feature["properties"]["name"]
    return ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8111)


In [ ]:
def transform_coordinates(geom, transformer):
    if geom.geom_type == 'Point':
        x, y = geom.x, geom.y
        return Point(*transformer.transform(x, y)[::-1])
    elif geom.geom_type == 'LineString':
        return LineString([transformer.transform(x, y)[::-1] for x, y in geom.coords])
    elif geom.geom_type == 'MultiLineString':
        return MultiLineString([
            LineString([transformer.transform(x, y)[::-1] for x, y in line.coords])
            for line in geom.geoms
        ])
    else:
        raise ValueError(f"Unsupported geometry type: {geom.geom_type}")

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def create_geojson(filtered_df):
    if filtered_df.empty:
        return []

    # Convert to GeoDataFrame for easier manipulation
    gdf = gpd.GeoDataFrame(filtered_df, geometry='line_geometry')

    # Convert to GeoJSON features
    features = []
    for index, row in gdf.iterrows():
        geom = row['line_geometry']

        if geom.is_empty:
            continue  # Skip empty geometries
        
        if isinstance(geom, LineString):
            lons, lats = map(list, geom.xy)  # Ensuring coordinates are lists
            features.append(go.Scattermapbox(
                lon=lons,
                lat=lats,
                mode='lines',
                line=dict(width=2),
                name=row.get('component_name', '')  # Example of using another column as name
            ))
        elif isinstance(geom, MultiLineString):
            for linestring in geom.geoms:  # Ensure geom.geoms is used to access individual LineStrings
                if isinstance(linestring, LineString):
                    lons, lats = map(list, linestring.xy)  # Ensuring coordinates are lists
                    features.append(go.Scattermapbox(
                        lon=lons,
                        lat=lats,
                        mode='lines',
                        line=dict(width=2),
                        name=row.get('component_name', '')  # Example of using another column as name
                    ))
                else:
                    print(f"Unexpected geometry within MultiLineString: {type(linestring)}")  # Unexpected type
        else:
            print(f"Unexpected geometry type: {type(geom)}")

    return features

app.layout = dbc.Container([
    dbc.Row(
        dbc.Col(html.H1("Component Analysis (Pre/Post)", className="app_title"),
                width=12),
                className="justify-content-center"
    ),
    dbc.Row([
        dbc.Col(
            [
                create_card("Completion year", "dropdown_year", sorted(vz_moped['completion_year'].unique()))
            ],
            className="navigation_row_col_style"
        ),
        dbc.Col(
            [
                create_card("Component name", "dropdown_component_name", vz_moped['component_name'].unique())
            ],
            className="navigation_row_col_style"
        ),
        dbc.Col(
            [
                create_card("Component sub-type", "dropdown_component_subtype", vz_moped['component_subtype'].unique())
            ],
            className="navigation_row_col_style"
        ),
        dbc.Col(
            [
                create_card("Component work-type", "dropdown_work_type", vz_moped['component_work_types'].unique())
            ],
            className="navigation_row_col_style"
        ),
        dbc.Col(
            [
                create_card("Fatal crash", "dropdown_fatal_crash", vz_moped['component_had_fatal_crash'].unique())
            ],
            className="navigation_row_col_style"
        )
    ], 
    className="navigation_row"),
    dbc.Row(
        [
            dbc.Col(
                dash_table.DataTable(
                    id='data_table',
                    columns=[{'name': column_labels[col], 'id': col} for col in columns_to_display],
                    data=vz_moped[columns_to_display].to_dict('records'),  # Ensure initial data is loaded
                    selected_row_ids=[],
                    style_table={'overflowX': 'auto', 'maxHeight': '600px'},  # Enable horizontal scrolling
                    style_cell={'minWidth': '150px', 'maxWidth': '200px', 'whiteSpace': 'normal'},  # Match style with external CSS
                    cell_selectable=True,
                    sort_action='native'
                ),
                width=8
            ),
            dbc.Col(dcc.Graph(id='map-plot'), width=4)
        ]
    )
], fluid=True, className="custom_container")


@app.callback(
    Output('data_table', 'data'),
    Output('map-plot', 'figure'),
    [
        Input('dropdown_year', 'value'),
        Input('dropdown_component_name', 'value'),
        Input('dropdown_component_subtype', 'value'),
        Input('dropdown_work_type', 'value'),
        Input('dropdown_fatal_crash', 'value')
    ]
)
def update_plot(dropdown_year, dropdown_component_name, dropdown_component_subtype,
                 dropdown_work_type, dropdown_fatal_crash):
    # Base DataFrame
    filtered_df = vz_moped

    # print(f"Dropdown Values: {dropdown_year}, {dropdown_component_name}, {dropdown_component_subtype}, {dropdown_work_type}, {dropdown_fatal_crash}")

    # Convert single values to list to avoid TypeError
    if dropdown_year and not isinstance(dropdown_year, list):
        dropdown_year = [dropdown_year]
    if dropdown_component_name and not isinstance(dropdown_component_name, list):
        dropdown_component_name = [dropdown_component_name]
    if dropdown_component_subtype and not isinstance(dropdown_component_subtype, list):
        dropdown_component_subtype = [dropdown_component_subtype]
    if dropdown_work_type and not isinstance(dropdown_work_type, list):
        dropdown_work_type = [dropdown_work_type]
    if dropdown_fatal_crash and not isinstance(dropdown_fatal_crash, list):
        dropdown_fatal_crash = [dropdown_fatal_crash]

    # Apply filters
    if dropdown_year:
        filtered_df = filtered_df[filtered_df['completion_year'].isin(dropdown_year)]
    if dropdown_component_name:
        filtered_df = filtered_df[filtered_df['component_name'].isin(dropdown_component_name)]
    if dropdown_component_subtype:
        filtered_df = filtered_df[filtered_df['component_subtype'].isin(dropdown_component_subtype)]
    if dropdown_work_type:
        filtered_df = filtered_df[filtered_df['component_work_types'].isin(dropdown_work_type)]
    if dropdown_fatal_crash:
        filtered_df = filtered_df[filtered_df['component_had_fatal_crash'].isin(dropdown_fatal_crash)]

    # print(f"Filtered Data: \n{filtered_df[columns_to_display]}")

    map_features = create_geojson(filtered_df)

    fig = go.Figure(go.Scattermapbox())

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=10,
        mapbox_center={"lat": 30.2672, "lon": -97.7431}  # Customize based on your default location
    )
    
    for feature in map_features:
        fig.add_trace(feature)

    return filtered_df[columns_to_display].to_dict('records'), fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8090)